In [1]:
import csv
import json
import logging
import os
from typing import IO, Optional, TextIO

import pandas as pd

from dadis_client.client import DadisClient

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='%(message)s')

In [2]:
API_KEY = os.environ["DADIS_API_KEY"]
client = DadisClient(api_key=API_KEY, prod=True)

VBO_BREED_FILENAME = "data/dadisbreedcountry.tsv"

In [3]:
def full_local_match_workflow(
    input_filename: str,
    output_filename: str,
    dadis_api_key: str) -> pd.DataFrame:
    """
    Perform the full matching workflow:

    - Read VBO data from input_filename
    - Match to DADIS to get DADIS ids
    - Save to a new TSV file at output_filename
    """
    client = DadisClient(api_key=dadis_api_key)
    logger.info(f"Reading VBO entries from {input_filename}")
    vbo_data = read_vbo_data(input_filename)
    logger.info(f"Matching to DADIS data")
    matched_breeds = match_vbo_breeds(vbo_data=vbo_data, client=client)

    logger.info(f"Writing output file to {output_filename}:")
    output_file = create_output_tsv(
        input_filename=input_filename, 
        output_filename=output_filename, 
        extra_cols=[
            "dadis_breed_id",
            "dadis_transboundary_id",
            "dadis_update_date",
        ]
    )
    matched_breeds.to_csv(output_file, sep="\t", index=False, header=False)
    output_file.close()
    logger.info("Output written.")
    return matched_breeds


def read_vbo_data(filename: str) -> pd.DataFrame:
    vbo_breeds = pd.read_table(filename, sep="\t", skiprows=[1], low_memory=False).convert_dtypes()
    return vbo_breeds


def get_dadis_species(client: DadisClient) -> pd.DataFrame:
    resp = client.get_all_species()
    all_species = []
    for s in resp.response:
        species = {"dadis_species_id": s.id, "dadis_species_name": s.name["en"]}
        all_species.append(species)
    return pd.DataFrame.from_records(all_species)


def get_dadis_all_breeds(client: DadisClient) -> pd.DataFrame:
    resp = client.get_all_breeds()
    df = (
        pd.DataFrame.from_records([
            breed.model_dump() for breed in resp.response
        ])
        .convert_dtypes()
        .rename(columns={
            "id": "dadis_breed_id",
            "name": "dadis_breed_name",
            "iso3": "dadis_iso3_code",
            "speciesId": "dadis_species_id",
            "transboundaryId": "dadis_transboundary_id",
            "updatedAt": "dadis_update_date"
        })
    )
    df["dadis_update_date"] = df["dadis_update_date"].map(
        lambda d: pd.to_datetime(d, unit="ms")
    )
    # Merge species information
    species_df = get_dadis_species(client)
    df = df.merge(species_df, how="left", on="dadis_species_id")
    return df

def match_vbo_breeds(vbo_data: pd.DataFrame, client: DadisClient) -> pd.DataFrame:
    """
    Match VBO breed entries to DADIS, based on breed name, species, and country (ISO3 code)
    """
    logger.info("Fetching DADIS breeds")
    dadis_all = get_dadis_all_breeds(client=client)
    merged = vbo_data.merge(
        dadis_all,
        how="left",
        left_on=["dadis_name", "dadis_species_name", "dadis_iso3_code"],
        right_on=["dadis_breed_name", "dadis_species_name", "dadis_iso3_code"],
        sort=False,
        indicator=True
    )
    n_matched = merged["_merge"].eq("both").sum()
    n_total = len(merged["_merge"])
    logger.info(f"{n_matched} / {n_total} VBO breeds successfully matched to DADIS IDs")
    merged = merged.drop(columns=["_merge", "dadis_breed_name", "dadis_species_id"])
    return merged

def find_unmatched_dadis(vbo_output: pd.DataFrame, client: DadisClient) -> pd.DataFrame:
    dadis_all = get_dadis_all_breeds(client)
    dadis_unmatched = (
        dadis_all
        .merge(vbo_output[["dadis_breed_id", "vbo_id"]], on="dadis_breed_id", how="left", indicator=True)
        .loc[lambda x: x._merge == "left_only"]
    )
    return dadis_unmatched

def create_output_tsv(input_filename: str, output_filename: str, extra_cols: list[str] = None) -> TextIO:
    """
    Copy the 2 header lines from the input file to the output file. Return
    a file object for the output file, so pandas can write the rest of the file
    """
    file_out = open(output_filename, "w")
    csv_out = csv.writer(file_out, dialect="excel-tab")
    with open(input_filename) as file_in:
        csv_in = csv.reader(file_in, dialect="excel-tab")
        for index, line in enumerate(range(2)):
            header = next(csv_in)
            if extra_cols is not None:
                if index == 0:
                    header += extra_cols
                if index == 1:
                    header += ['' for i in range(len(extra_cols))]
            csv_out.writerow(header)
    return file_out

## Test full workflow

In [4]:
output = full_local_match_workflow(
    input_filename=VBO_BREED_FILENAME,
    output_filename="example_local_breed_output.tsv",
    dadis_api_key=API_KEY
)

Reading VBO entries from data/dadisbreedcountry.tsv
Matching to DADIS data
Fetching DADIS breeds
14537 / 15143 VBO breeds successfully matched to DADIS IDs
Writing output file to example_local_breed_output.tsv:
Output written.


In [5]:
output.head()

,vbo_id,term_label,to_be_ignored,breed_name_to_be_used_for_label,species_to_be_used_for_label,internal_term_merge_indicator_vboid_into_vboid,internal_date_and_issue_separated_by_pipe_yymmdd_dash_issue,GH_issue_NOT_merge_obsolete,parent_ID,source_for_parents,...,GH_issue,obsolescence_reason,curator_note,dadis_name,dadis_country,dadis_species_name,dadis_iso3_code,dadis_breed_id,dadis_transboundary_id,dadis_update_date
0,VBO:0016872,"obsolete Alaska, United States of America (Nor...",<NA>,"Alaska, United States of America",North American deer mouse,<NA>,231211-135,https://github.com/monarch-initiative/vertebra...,<NA>,<NA>,...,https://github.com/monarch-initiative/vertebra...,domain entity does not exist,<NA>,Alaska,United States of America,North American deer mouse,USA,<NA>,<NA>,NaT
1,VBO:0016854,"obsolete Rocky Mountain, United States of Amer...",<NA>,"Rocky Mountain, United States of America",Bighorn sheep,<NA>,231211-135,https://github.com/monarch-initiative/vertebra...,<NA>,<NA>,...,https://github.com/monarch-initiative/vertebra...,domain entity does not exist,<NA>,Rocky Mountain,United States of America,Bighorn sheep,USA,<NA>,<NA>,NaT
2,VBO:0001723,"Alpakka, Finland (Alpaca)",<NA>,"Alpakka, Finland",Alpaca,<NA>,231213-139,https://github.com/monarch-initiative/vertebra...,VBO:0000038,https://www.fao.org/dad-is,...,<NA>,<NA>,<NA>,Alpakka,Finland,Alpaca,FIN,4036812f-308c-4b11-8402-eaf171a818a1,,2023-09-14 00:03:28
3,VBO:0001724,"Huacaya, Australia (Alpaca)",<NA>,"Huacaya, Australia",Alpaca,<NA>,<NA>,<NA>,VBO:0000039,https://www.fao.org/dad-is,...,<NA>,<NA>,<NA>,Huacaya,Australia,Alpaca,AUS,da0f5e4b-2dc4-4119-bb9a-6d87d88b2a3c,1-1,2022-05-10 08:48:11
4,VBO:0001725,"Huacaya, Bolivia (Plurinational State of) (Alp...",<NA>,"Huacaya, Bolivia (Plurinational State of)",Alpaca,<NA>,<NA>,<NA>,VBO:0000039,https://www.fao.org/dad-is,...,<NA>,<NA>,<NA>,Huacaya,Bolivia (Plurinational State of),Alpaca,BOL,1fb630e0-c8e7-4bfc-bbdb-d8d0db22538b,1-1,2022-10-12 14:02:06


### Other direction: which DADIS entries have a match?

In [6]:
dadis_all = get_dadis_all_breeds(client)
dadis_all.head()

,dadis_breed_id,dadis_breed_name,dadis_iso3_code,dadis_species_id,dadis_transboundary_id,dadis_update_date,dadis_species_name
0,ecf7d217-63a9-4b4f-aa6c-7afb49091604,Donkey,AFG,3,,2023-06-06 15:13:17,Ass
1,5be063e7-258a-48a8-bd8a-cb3f99bb5222,Afghan,AFG,7,,2023-06-06 15:05:18,Cattle
2,6c1cb9f9-e8b1-4127-b10f-c9cb4d312390,Kandahari,AFG,7,,2021-12-17 08:00:07,Cattle
3,e53309e3-8952-4b4a-864f-37ece264b93e,Konari,AFG,7,,2021-12-17 08:00:10,Cattle
4,00374759-f2aa-4943-9eb2-96e32f181869,Kunari,AFG,7,,2021-12-17 08:00:08,Cattle


In [7]:
dadis_unmatched = (
    dadis_all
    .merge(output[["dadis_breed_id", "vbo_id"]], on="dadis_breed_id", how="left", indicator=True)
    .loc[lambda x: x._merge == "left_only"]
)
dadis_unmatched.head()

,dadis_breed_id,dadis_breed_name,dadis_iso3_code,dadis_species_id,dadis_transboundary_id,dadis_update_date,dadis_species_name,vbo_id,_merge
325,1ce2ca66-e189-4895-b595-36d570b83ba3,Bison,AUS,2,,2022-10-28 04:18:30,American Bison,<NA>,left_only
332,0b6225aa-1fea-4284-a63a-fc6bca15f1e5,Alexandria Composite,AUS,7,,2022-11-02 03:47:33,Cattle,<NA>,left_only
334,ed0ded11-f8eb-4cef-ae0b-ee3c9adfce0f,Ankole Watusi,AUS,7,7-11,2022-11-09 22:02:28,Cattle,<NA>,left_only
358,c5d734d4-2f93-43d6-ae80-c71b1d446001,Corriente,AUS,7,7-68,2022-11-02 01:26:31,Cattle,<NA>,left_only
362,9556ad18-80f9-4eef-964c-11d68d44f592,Drakensberger,AUS,7,7-77,2022-11-09 22:02:30,Cattle,<NA>,left_only


# Step-by-step workflow for reference/debugging

## DADIS data

In [8]:
dadis_all = get_dadis_all_breeds(client)
dadis_all.head()

,dadis_breed_id,dadis_breed_name,dadis_iso3_code,dadis_species_id,dadis_transboundary_id,dadis_update_date,dadis_species_name
0,ecf7d217-63a9-4b4f-aa6c-7afb49091604,Donkey,AFG,3,,2023-06-06 15:13:17,Ass
1,5be063e7-258a-48a8-bd8a-cb3f99bb5222,Afghan,AFG,7,,2023-06-06 15:05:18,Cattle
2,6c1cb9f9-e8b1-4127-b10f-c9cb4d312390,Kandahari,AFG,7,,2021-12-17 08:00:07,Cattle
3,e53309e3-8952-4b4a-864f-37ece264b93e,Konari,AFG,7,,2021-12-17 08:00:10,Cattle
4,00374759-f2aa-4943-9eb2-96e32f181869,Kunari,AFG,7,,2021-12-17 08:00:08,Cattle


In [9]:
dadis_all.query('dadis_breed_name.str.contains("Huacaya")')

,dadis_breed_id,dadis_breed_name,dadis_iso3_code,dadis_species_id,dadis_transboundary_id,dadis_update_date,dadis_species_name
323,da0f5e4b-2dc4-4119-bb9a-6d87d88b2a3c,Huacaya,AUS,1,1-1,2022-05-10 08:48:11,Alpaca
1505,1fb630e0-c8e7-4bfc-bbdb-d8d0db22538b,Huacaya,BOL,1,1-1,2022-10-12 14:02:06,Alpaca
1556,78b28cda-58ec-4193-ac62-94a38063fbc2,Huacayas,BOL,23,,2021-12-17 08:09:22,Guinea pig
1987,242da190-ecdf-4b8e-beaf-355de1d2b0d5,Huacaya,CAN,1,1-1,2021-12-17 08:11:51,Alpaca
2262,6778e416-e0ac-42f7-9749-27edce9c1890,Huacaya,CHL,1,1-1,2022-10-12 15:02:29,Alpaca
11183,8a3a635d-0483-445a-a9ed-157d58146e25,Huacaya,PER,1,1-1,2022-10-13 01:02:08,Alpaca


## VBO breed data

In [10]:
vbo_breeds = pd.read_table(VBO_BREED_FILENAME, sep="\t", skiprows=[1], low_memory=False).convert_dtypes()
# TODO: temp fix for wrong/swapped column names
country = vbo_breeds["dadis_species_name"].copy()
species = vbo_breeds["dadis_country"].copy()
vbo_breeds["dadis_country"] = country
vbo_breeds["dadis_species_name"] = species
vbo_breeds.head()

,vbo_id,term_label,to_be_ignored,breed_name_to_be_used_for_label,species_to_be_used_for_label,internal_term_merge_indicator_vboid_into_vboid,internal_date_and_issue_separated_by_pipe_yymmdd_dash_issue,GH_issue_NOT_merge_obsolete,parent_ID,source_for_parents,...,replacement_term,replacement_label,comment.1,GH_issue,obsolescence_reason,curator_note,dadis_name,dadis_country,dadis_species_name,dadis_iso3_code
0,VBO:0016872,"obsolete Alaska, United States of America (Nor...",<NA>,"Alaska, United States of America",North American deer mouse,<NA>,231211-135,https://github.com/monarch-initiative/vertebra...,<NA>,<NA>,...,<NA>,<NA>,<NA>,https://github.com/monarch-initiative/vertebra...,domain entity does not exist,<NA>,Alaska,North American deer mouse,United States of America,USA
1,VBO:0016854,"obsolete Rocky Mountain, United States of Amer...",<NA>,"Rocky Mountain, United States of America",Bighorn sheep,<NA>,231211-135,https://github.com/monarch-initiative/vertebra...,<NA>,<NA>,...,<NA>,<NA>,<NA>,https://github.com/monarch-initiative/vertebra...,domain entity does not exist,<NA>,Rocky Mountain,Bighorn sheep,United States of America,USA
2,VBO:0001723,"Alpakka, Finland (Alpaca)",<NA>,"Alpakka, Finland",Alpaca,<NA>,231213-139,https://github.com/monarch-initiative/vertebra...,VBO:0000038,https://www.fao.org/dad-is,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Alpakka,Alpaca,Finland,FIN
3,VBO:0001724,"Huacaya, Australia (Alpaca)",<NA>,"Huacaya, Australia",Alpaca,<NA>,<NA>,<NA>,VBO:0000039,https://www.fao.org/dad-is,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Huacaya,Alpaca,Australia,AUS
4,VBO:0001725,"Huacaya, Bolivia (Plurinational State of) (Alp...",<NA>,"Huacaya, Bolivia (Plurinational State of)",Alpaca,<NA>,<NA>,<NA>,VBO:0000039,https://www.fao.org/dad-is,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Huacaya,Alpaca,Bolivia (Plurinational State of),BOL


In [11]:
# TODO: ignore obsolete terms? or will these be ignored anyway because the term label won't match?
match_data = vbo_breeds[["vbo_id", "term_label", "dadis_name", "dadis_species_name", "dadis_iso3_code"]]
match_data.head()

,vbo_id,term_label,dadis_name,dadis_species_name,dadis_iso3_code
0,VBO:0016872,"obsolete Alaska, United States of America (Nor...",Alaska,United States of America,USA
1,VBO:0016854,"obsolete Rocky Mountain, United States of Amer...",Rocky Mountain,United States of America,USA
2,VBO:0001723,"Alpakka, Finland (Alpaca)",Alpakka,Finland,FIN
3,VBO:0001724,"Huacaya, Australia (Alpaca)",Huacaya,Australia,AUS
4,VBO:0001725,"Huacaya, Bolivia (Plurinational State of) (Alp...",Huacaya,Bolivia (Plurinational State of),BOL


In [12]:
merged = match_data.merge(
    dadis_all,
    how="left",
    left_on=["dadis_name", "dadis_species_name", "dadis_iso3_code"],
    right_on=["dadis_breed_name", "dadis_species_name", "dadis_iso3_code"],
    sort=False,
    indicator=True
)
merged.head()

,vbo_id,term_label,dadis_name,dadis_species_name,dadis_iso3_code,dadis_breed_id,dadis_breed_name,dadis_species_id,dadis_transboundary_id,dadis_update_date,_merge
0,VBO:0016872,"obsolete Alaska, United States of America (Nor...",Alaska,United States of America,USA,<NA>,<NA>,<NA>,<NA>,NaT,left_only
1,VBO:0016854,"obsolete Rocky Mountain, United States of Amer...",Rocky Mountain,United States of America,USA,<NA>,<NA>,<NA>,<NA>,NaT,left_only
2,VBO:0001723,"Alpakka, Finland (Alpaca)",Alpakka,Finland,FIN,<NA>,<NA>,<NA>,<NA>,NaT,left_only
3,VBO:0001724,"Huacaya, Australia (Alpaca)",Huacaya,Australia,AUS,<NA>,<NA>,<NA>,<NA>,NaT,left_only
4,VBO:0001725,"Huacaya, Bolivia (Plurinational State of) (Alp...",Huacaya,Bolivia (Plurinational State of),BOL,<NA>,<NA>,<NA>,<NA>,NaT,left_only


In [13]:
n_matched = merged["_merge"].eq("both").sum()
n_matched
n_total = len(merged["_merge"])
n_total

15143

In [14]:
merged["_merge"].value_counts()

_merge
left_only     15141
both              2
right_only        0
Name: count, dtype: int64

In [15]:
merged.query('_merge == "left_only"')

,vbo_id,term_label,dadis_name,dadis_species_name,dadis_iso3_code,dadis_breed_id,dadis_breed_name,dadis_species_id,dadis_transboundary_id,dadis_update_date,_merge
0,VBO:0016872,"obsolete Alaska, United States of America (Nor...",Alaska,United States of America,USA,<NA>,<NA>,<NA>,<NA>,NaT,left_only
1,VBO:0016854,"obsolete Rocky Mountain, United States of Amer...",Rocky Mountain,United States of America,USA,<NA>,<NA>,<NA>,<NA>,NaT,left_only
2,VBO:0001723,"Alpakka, Finland (Alpaca)",Alpakka,Finland,FIN,<NA>,<NA>,<NA>,<NA>,NaT,left_only
3,VBO:0001724,"Huacaya, Australia (Alpaca)",Huacaya,Australia,AUS,<NA>,<NA>,<NA>,<NA>,NaT,left_only
4,VBO:0001725,"Huacaya, Bolivia (Plurinational State of) (Alp...",Huacaya,Bolivia (Plurinational State of),BOL,<NA>,<NA>,<NA>,<NA>,NaT,left_only
...,...,...,...,...,...,...,...,...,...,...,...
15138,VBO:0016870,"Japanese Draft, Japan (Horse)",Japanese Draft,Japan,JPN,<NA>,<NA>,<NA>,<NA>,NaT,left_only
15139,VBO:0016871,"Kentucky Mountain Saddle, United States of Ame...",Kentucky Mountain Saddle,United States of America,USA,<NA>,<NA>,<NA>,<NA>,NaT,left_only
15140,VBO:0016873,"Essex, United Kingdom of Great Britain and Nor...",Essex,United Kingdom of Great Britain and Northern I...,GBR,<NA>,<NA>,<NA>,<NA>,NaT,left_only
15141,VBO:0016874,"Minnesota, United States of America (Pig)",Minnesota,United States of America,USA,<NA>,<NA>,<NA>,<NA>,NaT,left_only


## Matching

In [16]:
matched = match_vbo_breeds(vbo_data=vbo_breeds, client=client)
matched.head()

Fetching DADIS breeds
2 / 15143 VBO breeds successfully matched to DADIS IDs


,vbo_id,term_label,to_be_ignored,breed_name_to_be_used_for_label,species_to_be_used_for_label,internal_term_merge_indicator_vboid_into_vboid,internal_date_and_issue_separated_by_pipe_yymmdd_dash_issue,GH_issue_NOT_merge_obsolete,parent_ID,source_for_parents,...,GH_issue,obsolescence_reason,curator_note,dadis_name,dadis_country,dadis_species_name,dadis_iso3_code,dadis_breed_id,dadis_transboundary_id,dadis_update_date
0,VBO:0016872,"obsolete Alaska, United States of America (Nor...",<NA>,"Alaska, United States of America",North American deer mouse,<NA>,231211-135,https://github.com/monarch-initiative/vertebra...,<NA>,<NA>,...,https://github.com/monarch-initiative/vertebra...,domain entity does not exist,<NA>,Alaska,North American deer mouse,United States of America,USA,<NA>,<NA>,NaT
1,VBO:0016854,"obsolete Rocky Mountain, United States of Amer...",<NA>,"Rocky Mountain, United States of America",Bighorn sheep,<NA>,231211-135,https://github.com/monarch-initiative/vertebra...,<NA>,<NA>,...,https://github.com/monarch-initiative/vertebra...,domain entity does not exist,<NA>,Rocky Mountain,Bighorn sheep,United States of America,USA,<NA>,<NA>,NaT
2,VBO:0001723,"Alpakka, Finland (Alpaca)",<NA>,"Alpakka, Finland",Alpaca,<NA>,231213-139,https://github.com/monarch-initiative/vertebra...,VBO:0000038,https://www.fao.org/dad-is,...,<NA>,<NA>,<NA>,Alpakka,Alpaca,Finland,FIN,<NA>,<NA>,NaT
3,VBO:0001724,"Huacaya, Australia (Alpaca)",<NA>,"Huacaya, Australia",Alpaca,<NA>,<NA>,<NA>,VBO:0000039,https://www.fao.org/dad-is,...,<NA>,<NA>,<NA>,Huacaya,Alpaca,Australia,AUS,<NA>,<NA>,NaT
4,VBO:0001725,"Huacaya, Bolivia (Plurinational State of) (Alp...",<NA>,"Huacaya, Bolivia (Plurinational State of)",Alpaca,<NA>,<NA>,<NA>,VBO:0000039,https://www.fao.org/dad-is,...,<NA>,<NA>,<NA>,Huacaya,Alpaca,Bolivia (Plurinational State of),BOL,<NA>,<NA>,NaT
